In [2]:
import pandas as pd 
df = pd.read_parquet(r'C:\Users\michael\OneDrive\Documentos\GitHub\Projeto-Cripto\Analises\202507\dados_prices_metrics\AAVE_15m_MACD.parquet')
df

,close_price,open_time,open_price,high_price,low_price,volume,formatted_time,acao_tomada,open_time_join,last_macd,last_signal,macd_hist
0,199.46,2024-11-29 18:00:00-03:00,199.27,199.77,199.02,688.858,2024-11-29 18:00,None,NaT,NaN,NaN,NaN
1,199.36,2024-11-29 18:15:00-03:00,199.44,199.44,198.64,419.738,2024-11-29 18:15,None,NaT,NaN,NaN,NaN
2,199.07,2024-11-29 18:30:00-03:00,199.32,199.46,198.87,504.134,2024-11-29 18:30,None,NaT,NaN,NaN,NaN
3,199.75,2024-11-29 18:45:00-03:00,199.07,199.75,198.79,489.706,2024-11-29 18:45,None,NaT,NaN,NaN,NaN
4,199.66,2024-11-29 19:00:00-03:00,199.71,200.34,199.61,604.493,2024-11-29 19:00,None,NaT,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
22364,326.08,2025-07-20 17:00:00-03:00,327.11,327.33,324.87,1773.889,2025-07-20 17:00,None,2025-07-20 17:00:00-03:00,-0.318406,0.117093,-0.435499
22365,326.50,2025-07-20 17:15:00-03:00,326.08,327.03,325.65,690.352,2025-07-20 17:15,None,2025-07-20 17:15:00-03:00,-0.393490,0.014976,-0.408466
22366,327.27,2025-07-20 17:30:00-03:00,326.49,327.47,326.27,620.462,2025-07-20 17:30,None,2025-07-20 17:30:00-03:00,-0.386407,-0.065300,-0.321107
22367,325.87,2025-07-20 17:45:00-03:00,327.23,327.86,325.80,917.066,2025-07-20 17:45,None,2025-07-20 17:45:00-03:00,-0.488135,-0.149867,-0.338268


In [3]:
df['acao_tomada'].value_counts()

acao_tomada
buy     909
sell    909
Name: count, dtype: int64

In [1]:
import numpy as np
import pandas as pd
from xgboost import XGBRegressor

# Simulação de série temporal
np.random.seed(42)
N = 1000
df = pd.DataFrame({'Y': np.random.randn(N).cumsum()})

# Indicador: média móvel
df['X1'] = df['Y'].rolling(window=3).mean().shift(1)

# Lags
df['Y_lag1'] = df['Y'].shift(1)
df['Y_lag2'] = df['Y'].shift(2)

df.dropna(inplace=True)

# Split em treino + último registro para previsão futura
train = df.iloc[:-1]
last_row = df.iloc[-1:]

X_train = train[['X1', 'Y_lag1', 'Y_lag2']]
y_train = train['Y']

# Treinamento
model = XGBRegressor()
model.fit(X_train, y_train)

# Forecast 3 steps ahead (iterativo):
forecast_steps = 3
predictions = []

current_Y_lag1 = last_row['Y'].values[0]
current_Y_lag2 = last_row['Y_lag1'].values[0]
current_X1 = last_row['X1'].values[0]

for step in range(forecast_steps):
    # Monta X para próximo passo
    X_next = pd.DataFrame({
        'X1': [current_X1],
        'Y_lag1': [current_Y_lag1],
        'Y_lag2': [current_Y_lag2]
    })
    
    # Faz previsão
    y_pred = model.predict(X_next)[0]
    predictions.append(y_pred)
    
    # Atualiza lags para próxima iteração
    current_Y_lag2 = current_Y_lag1
    current_Y_lag1 = y_pred
    current_X1 = np.mean([y_pred, current_Y_lag1, current_Y_lag2])  # Exemplo simples de atualização de média móvel "na mão"

print(predictions)


[18.4224, 20.103334, 20.534325]


In [3]:
df

,Y,X1,Y_lag1,Y_lag2
3,2.529168,0.620434,1.006138,0.358450
4,2.295015,1.297919,2.529168,1.006138
5,2.060878,1.943441,2.295015,2.529168
6,3.640091,2.295020,2.060878,2.295015
7,4.407525,2.665328,3.640091,2.060878
...,...,...,...,...
995,16.892123,18.068298,17.173223,18.174843
996,18.689809,17.413396,16.892123,17.173223
997,19.330652,17.585052,18.689809,16.892123
998,18.759473,18.304195,19.330652,18.689809
